# Improve runtime of HKY

Mamie Wang 20200616

We will try to improve the runtime of HKY similarity function by using the hamming distance function to compute the total mutation and subtract the other two transition rate to compute the transversion rate. 

We will compare the runtime using simulated sequences from a binary tree.

In [1]:
import sys, os

sys.path.append(os.path.join(os.path.dirname(sys.path[0]),'../spectral-tree-inference/spectraltree'))

import numpy as np
import utils
import generation
import reconstruct_tree
import dendropy
import scipy
import time
from itertools import product
import matplotlib.pyplot as plt

from dendropy.model.discrete import simulate_discrete_chars, Jc69, Hky85
from dendropy.calculate.treecompare import symmetric_difference
import character_matrix

In [58]:
def hamming_dist_missing_values(vals, missing_val = "-"):
    classnames, indices = np.unique(vals, return_inverse=True)
    num_arr = indices.reshape(vals.shape)
    hamming_matrix = scipy.spatial.distance.squareform(scipy.spatial.distance.pdist(num_arr, metric='hamming'))
    missing_array = (vals==missing_val)
    pdist_xor = scipy.spatial.distance.squareform(scipy.spatial.distance.pdist(missing_array, lambda u,v: np.sum(np.logical_xor(u,v))))
    pdist_or = scipy.spatial.distance.squareform(scipy.spatial.distance.pdist(missing_array, lambda u,v: np.sum(np.logical_or(u,v))))
    
    return (hamming_matrix*vals.shape[1] - pdist_xor) / (np.ones_like(hamming_matrix) * vals.shape[1] - pdist_or)


def HKY_similarity_matrix_missing_data(observations, classes=None, verbose = False):
    m, N = observations.shape
    if classes is None:
        classes = np.unique(observations)
    k = len(classes)
    # From Tamura, K., and M. Nei. 1993
    # for each pair of sequences, 
    # 1. estimate the average base frequency for pairs of sequences
    # 2. compute purine transition proportion P1 (A <-> G)
    # 3. compute pyrimidine transition proportion P2 (T <-> C)
    # 3. compute transversion proportion Q (A <-> C, A <-> T, G <-> C, G <-> T)

    if verbose: print("Computing the average base frequency for each pair of sequences...")
    g = {}
    
    not_missing = observations != "-"
    not_missing_sum = np.sum(not_missing, axis = 1) # # not missing for each taxon
    not_missing_pair = np.array([a + b for a, b in product(not_missing_sum, repeat = 2)]).reshape((m, m))
    
    for x in classes:
        obs_x = observations == x 
        g[x] = np.array([np.sum(np.hstack([a, b])) for a, b in product(obs_x, repeat = 2)]).reshape((m, m))
        g[x] = g[x] / not_missing_pair

    
    g["R"] = g["A"] + g["G"]
    g["Y"] = g["T"] + g["C"]
    
    # compute transition and transversion proportion
    if verbose: print("Computing transition and transversion proportion for each pair of sequences...")
        
    P_1 = np.zeros((m,m))
    P_2 = np.zeros((m,m))
    
    A = hamming_dist_missing_values(observations, missing_val = "-")
    
    for i in range(m):
        for j in range(i + 1, m):
            neither_missing = np.logical_and(not_missing[i,:], not_missing[j,:])
            a = observations[i,:][neither_missing]
            b = observations[j,:][neither_missing]
            
            A_G = np.mean(np.logical_and(a == "A", b == "G") + np.logical_and(a == "G", b == "A"))
            P_1[i, j] = P_1[j, i] = A_G
            
            C_T = np.mean(np.logical_and(a == "C", b == "T") + np.logical_and(a == "T", b == "C"))
            P_2[i, j] = P_2[j, i] = C_T
            
    Q = A - P_1 - P_2
    print("P", P_1, P_2)
    #print("Q", Q)
    # compute the similarity (formula 7)
    if verbose: print("Computing similarity matrix")
    R = (1 - g["R"]/(2 * g["A"] * g["G"]) * P_1 - 1 / (2 * g["R"]) * Q)
    Y = (1 - g["Y"]/(2 * g["T"] * g["C"]) * P_2 - 1 / (2 * g["Y"]) * Q)
    T = (1 - 1/(2 * g["R"] * g["Y"]) * Q)
    S = np.sign(R) * (np.abs(R))**(8 * g["A"] * g["G"] / g["R"])
    S *= np.sign(Y) * (np.abs(Y))**(8 * g["T"] * g["C"] / g["Y"])
    S *= np.sign(T) * (np.abs(T))**(8 * (g["R"] * g["Y"] - g["A"] * g["G"] * g["Y"] / g["R"] - g["T"] * g["C"] * g["R"] / g["Y"]))

    return S

In [ ]:
import copy

In [57]:
def HKY_similarity_matrix_missing_data2(observations, classes=None, verbose = False):
    m, N = observations.shape
    if classes is None:
        classes = np.unique(observations)
    k = len(classes)
    # From Tamura, K., and M. Nei. 1993
    # for each pair of sequences, 
    # 1. estimate the average base frequency for pairs of sequences
    # 2. compute purine transition proportion P1 (A <-> G)
    # 3. compute pyrimidine transition proportion P2 (T <-> C)
    # 3. compute transversion proportion Q (A <-> C, A <-> T, G <-> C, G <-> T)

    if verbose: print("Computing the average base frequency for each pair of sequences...")
    g = {}
    
    not_missing = observations != "-"
    not_missing_sum = np.sum(not_missing, axis = 1) # # not missing for each taxon
    not_missing_pair = np.array([a + b for a, b in product(not_missing_sum, repeat = 2)]).reshape((m, m))
    
    for x in classes:
        obs_x = observations == x 
        g[x] = np.array([np.sum(np.hstack([a, b])) for a, b in product(obs_x, repeat = 2)]).reshape((m, m))
        g[x] = g[x] / not_missing_pair

    
    g["R"] = g["A"] + g["G"]
    g["Y"] = g["T"] + g["C"]
    
    # compute transition and transversion proportion
    if verbose: print("Computing transition and transversion proportion for each pair of sequences...")
        
    P_1 = np.zeros((m,m))
    P_2 = np.zeros((m,m))
    
    A = hamming_dist_missing_values(observations, missing_val = "-")
    
    A_G_bool = np.full_like(observations, "-")
    A_G_bool[observations == "A"] = "A"
    A_G_bool[observations == "G"] = "G"
    P_1 = hamming_dist_missing_values(A_G_bool, missing_val = "-")
    
    
    C_T_bool = np.full_like(observations, "-")
    C_T_bool[observations == "C"] = "C"
    C_T_bool[observations == "T"] = "T"
    P_2 = hamming_dist_missing_values(C_T_bool, missing_val = "-")
    
            
    Q = A - P_1 - P_2
    print("P", P_1, P_2)
    #print("Q", Q)
    # compute the similarity (formula 7)
    if verbose: print("Computing similarity matrix")
    R = (1 - g["R"]/(2 * g["A"] * g["G"]) * P_1 - 1 / (2 * g["R"]) * Q)
    Y = (1 - g["Y"]/(2 * g["T"] * g["C"]) * P_2 - 1 / (2 * g["Y"]) * Q)
    T = (1 - 1/(2 * g["R"] * g["Y"]) * Q)
    S = np.sign(R) * (np.abs(R))**(8 * g["A"] * g["G"] / g["R"])
    S *= np.sign(Y) * (np.abs(Y))**(8 * g["T"] * g["C"] / g["Y"])
    S *= np.sign(T) * (np.abs(T))**(8 * (g["R"] * g["Y"] - g["A"] * g["G"] * g["Y"] / g["R"] - g["T"] * g["C"] * g["R"] / g["Y"]))

    return S

In [49]:
m = 512
binary_tree = utils.balanced_binary(m, edge_length = 0.5)
data_HKY = simulate_discrete_chars(1000, binary_tree, Hky85(kappa = 1), mutation_rate=0.1)

ch_list = list()
for t in data_HKY.taxon_namespace:
    ch_list.append([x.symbol for x in data_HKY[t]])
ch_arr = np.array(ch_list)

In [5]:
start_time = time.time()
mat_old = reconstruct_tree.HKY_similarity_matrix_missing_data(ch_arr)
print(time.time() - start_time)

34.26752519607544


In [59]:
start_time = time.time()
mat_new = HKY_similarity_matrix_missing_data(ch_arr)
print(time.time() - start_time)

P [[0.    0.011 0.036 ... 0.102 0.102 0.1  ]
 [0.011 0.    0.036 ... 0.101 0.1   0.097]
 [0.036 0.036 0.    ... 0.102 0.099 0.095]
 ...
 [0.102 0.101 0.102 ... 0.    0.036 0.038]
 [0.102 0.1   0.099 ... 0.036 0.    0.016]
 [0.1   0.097 0.095 ... 0.038 0.016 0.   ]] [[0.    0.014 0.024 ... 0.083 0.086 0.079]
 [0.014 0.    0.025 ... 0.08  0.09  0.083]
 [0.024 0.025 0.    ... 0.082 0.086 0.079]
 ...
 [0.083 0.08  0.082 ... 0.    0.027 0.032]
 [0.086 0.09  0.086 ... 0.027 0.    0.017]
 [0.079 0.083 0.079 ... 0.032 0.017 0.   ]]
25.264612197875977


In [60]:
start_time = time.time()
mat_new2 = HKY_similarity_matrix_missing_data2(ch_arr)
print(time.time() - start_time)

P [[0.         0.02226721 0.07563025 ... 0.27868852 0.28021978 0.27777778]
 [0.02226721 0.         0.07775378 ... 0.28450704 0.28328612 0.27556818]
 [0.07563025 0.07775378 0.         ... 0.28813559 0.28448276 0.27536232]
 ...
 [0.27868852 0.28450704 0.28813559 ... 0.         0.07627119 0.08119658]
 [0.28021978 0.28328612 0.28448276 ... 0.07627119 0.         0.03285421]
 [0.27777778 0.27556818 0.27536232 ... 0.08119658 0.03285421 0.        ]] [[0.         0.0324826  0.05783133 ... 0.28040541 0.28289474 0.26072607]
 [0.0324826  0.         0.05995204 ... 0.26666667 0.29220779 0.26774194]
 [0.05783133 0.05995204 0.         ... 0.27242525 0.28196721 0.25901639]
 ...
 [0.28040541 0.26666667 0.27242525 ... 0.         0.06428571 0.07637232]
 [0.28289474 0.29220779 0.28196721 ... 0.06428571 0.         0.03794643]
 [0.26072607 0.26774194 0.25901639 ... 0.07637232 0.03794643 0.        ]]
15.250773906707764


In [51]:
np.sum(mat_new - mat_old > 1e-10)

0

In [56]:
np.sum(mat_new2 - mat_old > 1e-10)

3658